In [1]:
import pathlib, re, unicodedata
import numpy as np, pandas as pd, plotly.graph_objects as go

# --- 1) Paths --------------------------------------------------------------
ROOT_DIR  = pathlib.Path(r"C:\Repositories\odi-data-visualization")
DATA_FILE = ROOT_DIR / "data" / "table_3.csv"
PLOTS_DIR = ROOT_DIR / "plots"
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# --- 2) Safe name + normalization helpers ---------------------------------
# keep apostrophes; replace only truly illegal filename chars on Windows
SAFE  = re.compile(r"[^0-9A-Za-z _()\-.’']+")
def normalize_quotes(s: str) -> str:
    s = unicodedata.normalize("NFKC", str(s))
    return s.replace("’", "'").strip()

def clean(s: str, n=80) -> str:
    s = normalize_quotes(s)
    s = SAFE.sub("_", s)           # strip bad chars
    s = re.sub(r"\s+", " ", s)     # collapse spaces
    return s[:n].strip()

# --- 3) Gauge bands (scale 1–5) -------------------------------------------
BANDS = [
    (1.00, 3.29, "#e53935"),   # red
    (3.30, 3.49, "#fb8c00"),   # orange
    (3.50, 3.79, "#fdd835"),   # yellow
    (3.80, 5.00, "#43a047"),   # green
]
def band_color(v, default="#999999"):
    for lo, hi, col in BANDS:
        if lo <= v <= hi:
            return col
    return default

# --- 4) Load table ---------------------------------------------------------
df = pd.read_csv(DATA_FILE)

metric_names = df.iloc[:, 0].astype(str)      # first col = metric name
departments  = df.columns[1:]                 # the rest are departments

# Ensure numeric
for col in departments:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# --- 5) Group mapping (canonical slugs for folders) -----------------------
ELECTED_DEPTS = {
    "the city auditor", "city auditor",
    "city attorney",
    "city council"
}
def group_of(dept: str) -> str:
    norm = normalize_quotes(dept).casefold().strip()
    return "elected_officials" if norm in ELECTED_DEPTS else "mayors_office"

PRETTY_GROUP = {"elected_officials": "Elected Officials",
                "mayors_office": "Mayor’s Office"}

# --- 6) Gauge builder (ticks 1..5, 0.5 step, label integers) --------------
def build_gauge(score, title):
    vals = np.arange(1, 5.01, 0.5)
    txt  = [f"{v:.0f}" if float(v).is_integer() else "" for v in vals]
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=score,
        title={"text": title, "font": {"size": 18, "color": "#1f3d5a"}},
        number={"font": {"size": 38, "color": "#1f3d5a"}},
        gauge={
            "axis": {"range": [1, 5],
                     "tickmode": "array",
                     "tickvals": vals,
                     "ticktext": txt,
                     "ticks": "outside",
                     "ticklen": 8,
                     "tickwidth": 2,
                     "tickcolor": "#444",
                     "tickfont": {"size": 12}},
            "bar": {"color": band_color(score), "thickness": 0.45},
            "bgcolor": "#FFFFFF",
            "borderwidth": 1,
            "bordercolor": "#444"
        }
    ))
    fig.update_layout(height=450, width=450,
                      margin=dict(t=60, b=20, l=20, r=20))
    return fig

def save_png(fig, dirpath: pathlib.Path, filename: str):
    dirpath.mkdir(parents=True, exist_ok=True)
    fig.write_image(dirpath / f"{filename}.png", scale=2)

# --- 7) Helper: build prefix map for Top 3 / Bottom 3 ---------------------
def rank_prefixes(scores: pd.Series, n=3) -> dict:
    """
    Given a Series indexed by metric name, return a dict {metric: prefix}
    where prefix is '1_', '2_', '3_' for top n, and '-1_', '-2_', '-3_' for bottom n.
    If a metric is neither top nor bottom, it's omitted (no prefix).
    """
    s = scores.dropna()
    # keep stable order for ties
    top_idx = s.sort_values(ascending=False, kind="mergesort").head(n).index.tolist()
    bot_idx = s.sort_values(ascending=True,  kind="mergesort").head(n).index.tolist()
    pref = {}
    for i, m in enumerate(top_idx, 1):  pref[m] = f"{i}_"
    for i, m in enumerate(bot_idx, 1):  pref[m] = f"-{i}_"
    return pref

# --- 8) Department-level gauges (ALL metrics, filenames marked if top/btm) -
print("Generating department gauges…")
for dept in departments:
    grp_slug = group_of(dept)                 # 'elected_officials' | 'mayors_office'
    dept_dir = (PLOTS_DIR / grp_slug / clean(dept))
    # build the prefix map once per department
    s = pd.Series(df[dept].values, index=metric_names).dropna()
    prefixes = rank_prefixes(s, n=3)

    for metric, score in zip(metric_names, df[dept]):
        if pd.isna(score):
            continue
        fig = build_gauge(score, f"{dept} — {metric}")
        prefix = prefixes.get(metric, "")
        filename = prefix + clean(metric)
        save_png(fig, dept_dir, filename)
        print(f"   • {grp_slug} / {dept} — {metric}  →  {filename}.png")

# --- 9) Group-level averages (filenames marked if top/btm) ----------------
print("\nGenerating group-level gauges…")
for grp_slug in ["elected_officials", "mayors_office"]:
    grp_depts = [d for d in departments if group_of(d) == grp_slug]
    if not grp_depts:
        continue
    base_grp_dir = PLOTS_DIR / grp_slug

    # average score per metric across the group's departments
    rows = []
    for metric in metric_names:
        vals = pd.to_numeric(df.loc[metric_names == metric, grp_depts].values.flatten(),
                             errors="coerce")
        vals = vals[~np.isnan(vals)]
        if len(vals):
            rows.append((metric, float(vals.mean())))
    if not rows:
        continue

    gseries = pd.Series({m: s for m, s in rows})
    prefixes = rank_prefixes(gseries, n=3)

    for metric, avg_score in gseries.items():
        fig = build_gauge(avg_score, f"{PRETTY_GROUP[grp_slug]} — {metric}")
        filename = prefixes.get(metric, "") + clean(metric)
        save_png(fig, base_grp_dir, filename)
        print(f"   • {grp_slug} — {metric}  →  {filename}.png")

print("\nDone.  Plots saved under:")
print("   ", PLOTS_DIR / "elected_officials")
print("   ", PLOTS_DIR / "mayors_office")

Generating department gauges…
   • mayors_office / Building and Zoning Services — Overall  →  Overall.png
   • mayors_office / Building and Zoning Services — Training  →  3_Training.png
   • mayors_office / Building and Zoning Services — Supervisor Clarity/ Communication  →  1_Supervisor Clarity_ Communication.png
   • mayors_office / Building and Zoning Services — Communication with Upper Management  →  -3_Communication with Upper Management.png
   • mayors_office / Building and Zoning Services — Task Significance  →  2_Task Significance.png
   • mayors_office / Building and Zoning Services — Autonomy  →  Autonomy.png
   • mayors_office / Building and Zoning Services — Feedback/Recognition  →  Feedback_Recognition.png
   • mayors_office / Building and Zoning Services — Policies  →  Policies.png
   • mayors_office / Building and Zoning Services — Innovation  →  -1_Innovation.png
   • mayors_office / Building and Zoning Services — Satisfaction with my Coworkers/ Department Culture  →  S